In [1]:
import os
import json
import pandas as pd
from pathlib import Path

# plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# scoot functions
from cleanair.scoot import (
    generate_fp,
    load_model_from_file,
    load_processed_data_from_file,
    plotly_results
)

In [2]:
name = "test"

# make the figure directory
user_settings_fp = os.path.join("..", "..", "terraform", ".secrets", "user_settings.json")
with open(user_settings_fp) as json_file:
    user_settings = json.load(json_file)
xpfp = user_settings["experiment_dir"]  # root to experiments filepaths directory
figure_dir = os.path.join(xpfp, name, "figures")
settings_dir = os.path.join(xpfp, name, "settings")
Path(figure_dir).mkdir(exist_ok=True)

# get the settings for kernels and scoot data
with open(os.path.join(settings_dir, "kernel_settings.json")) as kernel_file:
    kernel_settings = json.load(kernel_file)
with open(os.path.join(settings_dir, "scoot_settings.json")) as scoot_file:
    scoot_settings = json.load(scoot_file)

# load the raw csv
normal_df = pd.read_csv(generate_fp(name, xp_root=xpfp))
normal_df = normal_df.loc[
    (normal_df.measurement_start_utc >= scoot_settings["normal_start"]) &
    (normal_df.measurement_start_utc < scoot_settings["normal_end"])
]
lockdown_df = pd.read_csv(generate_fp(name, prefix="lockdown", xp_root=xpfp))
lockdown_df = lockdown_df.loc[
    (lockdown_df.measurement_start_utc >= scoot_settings["lockdown_start"]) &
    (lockdown_df.measurement_start_utc < scoot_settings["lockdown_end"])
]

In [3]:
detector_id = "N00/002e1"
prefix = "normal"           # normal or lockdown
kernel_id = "periodicXrbf"
sensor_df = normal_df.loc[normal_df.detector_id==detector_id]

# load data for this prefix and detector
model = load_model_from_file(name, kernel_id, detector_id, prefix=prefix, xp_root=xpfp)
X, Y = load_processed_data_from_file(name, detector_id, prefix=prefix, xp_root=xpfp)

In [4]:
data = plotly_results(sensor_df, detector_id, model, X, num_samples=1000)
layout = go.Layout(
    title='Timeseries of sensor {id}'.format(id=detector_id),
    xaxis_title="Datetime",
    yaxis_title="# of vechicles per hour",
    font=dict(size=16)
)
fig = go.Figure(data=data, layout=layout)
fig.show()

In [25]:

detector_list = scoot_settings["scoot_ids"]
kernel_list = list(kernel_settings.keys())
num_cols = len(kernel_list)     # kernels on columns
num_rows = len(detector_list)   # detectors on rows

import itertools
titles = [d + " with " + str(k) + " kernel" for d, k in itertools.product(detector_list, kernel_list)]

fig = make_subplots(
    rows=num_rows, cols=num_cols, subplot_titles=titles, shared_xaxes=False, shared_yaxes=True
)

for i in range(num_rows):
    detector_id = detector_list[i]
    sensor_df = normal_df.loc[normal_df.detector_id==detector_id]
    for j in range(num_cols):
        kernel_id = kernel_list[j]
        k = i * num_cols + j  # index of borough in name list
        if k == len(titles):
            break

        model = load_model_from_file(name, kernel_id, detector_id, prefix=prefix, xp_root=xpfp)
        X, Y = load_processed_data_from_file(name, detector_id, prefix=prefix, xp_root=xpfp)

        data = plotly_results(sensor_df, detector_id, model, X, num_samples=1000)
        for d in data:
            fig.add_trace(d, row=i+1, col=j+1)
        
        # fig.add_trace(go.Scatter(
        #     x=b_df["measurement_start_utc"], y=b_df[col]
        # ), row=i+1, col=j+1)

In [26]:
fig.update_layout(
    width=1500,
    height=1000
)
fig.show()